In [5]:
#method to get frame from taking multiple videos stored in a folder only if there is a changement. detect automaticaaly if there is a changement of scene and then store in a folder the frame images for each scen of the video.
import os
import cv2
from scenedetect import VideoManager, SceneManager
from scenedetect.detectors import ContentDetector
from multiprocessing import Pool


def detect_and_save_frames(video_folder, output_folder, threshold=30.0, resize_factor=0.5):
    """
    Detects scene changes in videos using a proxy (lower resolution) version of the video.
    Saves all frames corresponding to each detected scene.

    Parameters:
    video_folder (str): Path to the folder containing video files.
    output_folder (str): Path to the folder to store extracted scene frames.
    threshold (float): Threshold for detecting scene changes. Higher values mean less sensitivity.
    resize_factor (float): Factor to downscale the resolution for proxy detection.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Process each video in the folder
    vid_num =0
    for video_file in os.listdir(video_folder):
        if not video_file.endswith(('.mp4', '.avi', '.mov', '.mkv')):
            continue

        video_path = os.path.join(video_folder, video_file)
        video_name = os.path.splitext(video_file)[0]
        video_output_folder = os.path.join(output_folder, video_name)#os.path.join(output_folder, str(vid_num))#

        if not os.path.exists(video_output_folder):
            os.makedirs(video_output_folder)

        # Use scenedetect to detect scenes
        video_manager = VideoManager([video_path])
        scene_manager = SceneManager()
        scene_manager.add_detector(ContentDetector(threshold=threshold))

        video_manager.start()
        scene_manager.detect_scenes(frame_source=video_manager)

        # Get list of scene changes
        scene_list = scene_manager.get_scene_list()

        # Open video with OpenCV for frame extraction
        cap = cv2.VideoCapture(video_path)
        original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        resized_width = int(original_width * resize_factor)
        resized_height = int(original_height * resize_factor)

        for i, (start_time, end_time) in enumerate(scene_list):
            scene_folder = os.path.join(video_output_folder, f"scene_{i + 1}")
            if not os.path.exists(scene_folder):
                os.makedirs(scene_folder)
            frame_num = 0
            for frame_idx in range(start_time.get_frames(), end_time.get_frames()):
                if frame_idx % 5 != 0:  # prendre  frame
                    continue  
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
                success, frame = cap.read()

                if success:
                    # Downscale frame for faster processing
                    resized_frame = cv2.resize(frame, (resized_width, resized_height))

                    # Skip black frames
                    gray_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
                    if cv2.countNonZero(gray_frame) < 0.01 * gray_frame.size:  # Skip frames that are mostly black
                        continue

                    # Save original-sized frame
                    frame_path = os.path.join(scene_folder, f"frame_{frame_num}.jpg")
                    cv2.imwrite(frame_path, frame)
                    frame_num += 1

        cap.release()
        video_manager.release()
        vid_num +=1

detect_and_save_frames("dataset/videos", "dataset/frame_from_videos")

VideoManager is deprecated and will be removed.


KeyboardInterrupt: 

In [ ]:
"""#method to get frame from taking multiple videos stored in a folder only if there is a changement. detect automaticaaly if there is a changement of scene and then store in a folder the frame images for each scen of the video.

from scenedetect import VideoManager, SceneManager
from scenedetect.detectors import ContentDetector


def detect_and_save_frames(video_folder, output_folder, threshold=30.0, resize_factor=0.5):
    """
   """ Detects scene changes in videos using a proxy (lower resolution) version of the video.
    Saves all frames corresponding to each detected scene.

    Parameters:
    video_folder (str): Path to the folder containing video files.
    output_folder (str): Path to the folder to store extracted scene frames.
    threshold (float): Threshold for detecting scene changes. Higher values mean less sensitivity.
    resize_factor (float): Factor to downscale the resolution for proxy detection."""
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Process each video in the folder
    vid_num =0
    for video_file in os.listdir(video_folder):
        if not video_file.endswith(('.mp4', '.avi', '.mov', '.mkv')):
            continue

        video_path = os.path.join(video_folder, video_file)
        video_name = os.path.splitext(video_file)[0]
        video_output_folder = os.path.join(output_folder, vid_num)#os.path.join(output_folder, video_name)

        if not os.path.exists(video_output_folder):
            os.makedirs(video_output_folder)

        # Use scenedetect to detect scenes
        video_manager = VideoManager([video_path])
        scene_manager = SceneManager()
        scene_manager.add_detector(ContentDetector(threshold=threshold))

        video_manager.start()
        scene_manager.detect_scenes(frame_source=video_manager)

        # Get list of scene changes
        scene_list = scene_manager.get_scene_list()

        # Open video with OpenCV for frame extraction
        cap = cv2.VideoCapture(video_path)
        original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        resized_width = int(original_width * resize_factor)
        resized_height = int(original_height * resize_factor)

        for i, (start_time, end_time) in enumerate(scene_list):
            scene_folder = os.path.join(video_output_folder, f"scene_{i + 1}")
            if not os.path.exists(scene_folder):
                os.makedirs(scene_folder)

            for frame_idx in range(start_time.get_frames(), end_time.get_frames()):
                if frame_idx % 5 != 0:  # prendre  frame
                    continue  
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
                success, frame = cap.read()

                if success:
                    # Downscale frame for faster processing
                    resized_frame = cv2.resize(frame, (resized_width, resized_height))

                    # Skip black frames
                    gray_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
                    if cv2.countNonZero(gray_frame) < 0.01 * gray_frame.size:  # Skip frames that are mostly black
                        continue

                    # Save original-sized frame
                    frame_path = os.path.join(scene_folder, f"frame_{frame_idx}.jpg")
                    cv2.imwrite(frame_path, frame)

        cap.release()
        video_manager.release()
        vid_num +=1

detect_and_save_frames("dataset/videos", "dataset/frame_from_videos")

"""